base.html

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Projekt Flask</title>
    <link rel="stylesheet" href="https://cdn.simplecss.org/simple.min.css">
</head>
<body>
    <header>
        <h1>Analiza danych</h1>
        {% if current_user.is_authenticated %}
        <p>Zalogowany jako {{ current_user.username }} | <a href="{{ url_for('main.logout') }}">Wyloguj</a></p>
        {% endif %}
    </header>
    <main>
        {% block content %}{% endblock %}
    </main>
</body>
</html>


dashboard.html

In [ ]:
{% extends 'base.html' %}
{% block content %}
<h2>Witaj, {{ name }}!</h2>
<p>Poniżej znajduje się analiza danych:</p>
<img src="{{ url_for('static', filename='plot.png') }}" alt="Wykres analizy">
<div class="card">
  <h3>Liczba ludności w Polsce (2015–2020)</h3>
  <img src="{{ url_for('static', filename='forecast.png') }}" alt="Wykres ludności" style="max-width:100%;">
</div>
{% endblock %}


login.html

In [ ]:
{% extends 'base.html' %}

{% block content %}

{% with messages = get_flashed_messages(with_categories=true) %}
  {% if messages %}
    {% for category, message in messages %}
      <div class="alert alert-{{ category }}">
        {{ message }}
      </div>
    {% endfor %}
  {% endif %}
{% endwith %}


<h2>Logowanie</h2>
<form method="POST">
    {{ form.hidden_tag() }}
    <p>
        {{ form.username.label }}<br>
        {{ form.username(size=32) }}
    </p>
    <p>
        {{ form.password.label }}<br>
        {{ form.password(size=32) }}
    </p>
    <p>
        {{ form.submit() }}
    </p>
</form>
{% endblock %}



register.html

In [ ]:
{% extends 'base.html' %}

{% block content %}
  <h2>Rejestracja</h2>
  <form method="POST">
    {{ form.hidden_tag() }}
    <div>
      {{ form.username.label }}<br>
      {{ form.username(size=32) }}
    </div>
    <div>
      {{ form.password.label }}<br>
      {{ form.password(size=32) }}
    </div>
    <div>
      {{ form.submit() }}
    </div>
  </form>
{% endblock %}


__init__.py

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager

db = SQLAlchemy()
login_manager = LoginManager()

def create_app():
    app = Flask(__name__)
    app.config['SECRET_KEY'] = 'a8b0f5a2d9e641fa8e0c324ef891e404'
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'

    db.init_app(app)
    login_manager.init_app(app)
    login_manager.login_view = 'login'

    from .routes import main
    app.register_blueprint(main)

    with app.app_context():
        db.create_all()

    return app


analysis.py

In [ ]:
import os
import pandas as pd
import matplotlib
matplotlib.use('Agg')  
import matplotlib.pyplot as plt

def generate_analysis_plot():

    static_folder = os.path.join(os.path.dirname(__file__), 'static')
    os.makedirs(static_folder, exist_ok=True)
    plot_path = os.path.join(static_folder, 'plot.png')

    data = {
        'Rok': [2015, 2016, 2017, 2018, 2019, 2020],
        'Liczba ludności': [38400, 38500, 38450, 38300, 38200, 38100]
    }
    df = pd.DataFrame(data)

    # Tworzenie wykresu
    plt.figure(figsize=(6, 4))
    plt.plot(df['Rok'], df['Liczba ludności'], marker='o')
    plt.title('Liczba ludności w Polsce (2015–2020)')
    plt.xlabel('Rok')
    plt.ylabel('Liczba ludności (tys.)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close()

def generate_forecast_plot():
    plot_path = os.path.join(os.path.dirname(__file__), 'static', 'forecast.png')

    data = {
        'Rok': [2023, 2030, 2040, 2050, 2060],
        '0–17': [18, 17, 16, 15, 14],
        '18–64': [63, 60, 56, 53, 51],
        '65+': [19, 23, 28, 32, 35]
    }
    df = pd.DataFrame(data)

    plt.figure(figsize=(6, 4))
    plt.stackplot(df['Rok'], df['0–17'], df['18–64'], df['65+'],
                  labels=['0–17', '18–64', '65+'], colors=['#90caf9', '#a5d6a7', '#ffcc80'])
    plt.title('Prognoza struktury wiekowej ludności (2023–2060)')
    plt.xlabel('Rok')
    plt.ylabel('Udział procentowy (%)')
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close()

forms.py

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import InputRequired, Length

class RegisterForm(FlaskForm):
    username = StringField('Username', validators=[InputRequired(), Length(min=4, max=15)])
    password = PasswordField('Password', validators=[InputRequired(), Length(min=4, max=80)])
    submit = SubmitField('Register') 

class LoginForm(FlaskForm):
    username = StringField('Username', validators=[InputRequired(), Length(min=4, max=15)])
    password = PasswordField('Password', validators=[InputRequired(), Length(min=4, max=80)])
    submit = SubmitField('Login')  


models.py

In [ ]:
from . import db
from flask_login import UserMixin

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(150), unique=True, nullable=False)
    password = db.Column(db.String(150), nullable=False)


routes.py

In [ ]:
from flask import Blueprint, render_template, redirect, url_for, request
from .forms import RegisterForm, LoginForm
from .models import User
from . import db
from flask_login import login_user, logout_user, login_required, current_user
from werkzeug.security import generate_password_hash, check_password_hash
from .analysis import generate_analysis_plot, generate_forecast_plot
from flask_login import LoginManager

main = Blueprint('main', __name__)

from . import login_manager

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@main.route('/')
def index():
    return redirect(url_for('main.login'))

@main.route('/register', methods=['GET', 'POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_pw = generate_password_hash(form.password.data, method='pbkdf2:sha256')
        new_user = User(username=form.username.data, password=hashed_pw)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('main.login'))
    return render_template('register.html', form=form)

@main.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user and check_password_hash(user.password, form.password.data):
            login_user(user)
            return redirect(url_for('main.dashboard'))
    return render_template('login.html', form=form)

@main.route('/dashboard')
@login_required
def dashboard():
    generate_analysis_plot()
    generate_forecast_plot()
    return render_template('dashboard.html', name=current_user.username)

@main.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('main.login'))



run.py

In [ ]:
from app import create_app

app = create_app()

if __name__ == '__main__':
    app.run(debug=True)
